# Data Prepatation



In [2]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import json
from config import CLIENT_ID, CLIENT_SECRET


### Download and preprocess data

In [3]:
def clean_sheets(sheets):
    '''
    Get rid of the duplicate sheets
    Only take the sheets from the latest point in the day
    '''
    
    new_ranges = []
    for s in sheets:
        new_ranges.append(s.get("properties", {}).get("title"))
        
    clean_new_ranges = new_ranges.copy()
    for i, x in enumerate(clean_new_ranges):
        clean_new_ranges[i] = x.split('_')[0]

    indices = []
    for item in set(clean_new_ranges):
        indices.append(clean_new_ranges.index(item))

    clean_new_ranges = []
    for index in sorted(indices):
        clean_new_ranges.append(new_ranges[index])

    return clean_new_ranges
    

In [4]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w'

"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

#get all the sheet names for ranges when querying
sheet_metadata = service.spreadsheets().get(spreadsheetId=SAMPLE_SPREADSHEET_ID).execute()
sheets = sheet_metadata.get('sheets', '')

cleaned_ranges = clean_sheets(sheets)

In [5]:
'''
Now that we have the sheet names cleaned
we can now start doing some extractions
'''
def get_col_names():
    col_names= []
    for sheet_range in cleaned_ranges:
        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=sheet_range).execute()
        values = result.get('values', [])      
        for col in values[0]:
            col_names.append(col[:])
    return set(col_names)

cols = get_col_names()

In [6]:
def get_data(sheet_range):
    tmp_df = pd.DataFrame([])
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=sheet_range).execute()

    header = result.get('values', [])[0]   # Assumes first line is header!
    values = result.get('values', [])[1:]  # Everything else is data.
    
    '''
    rows with no deaths and recovered vals have shorter lists
    '''
    for i, row in enumerate(values):
        if len(row) < len(header):
            extra_zeros = (len(header) - len(row))
            values[i] += [0] * extra_zeros

    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        tmp = pd.concat(all_data, axis=1)
        
    print(sheet_range)
    return tmp
   
df_list = []
for sheet_range in cleaned_ranges:
    df_list.append(get_data(sheet_range))

Jan29_230pm
Jan28_11pm
Jan27_830pm
Jan26_11pm
Jan25_10pm
Jan24_12pm
Jan23_12pm
Jan22_12pm


Now that we have all the data we now need to clean it 
- Fill null values
- remore suspected values
- change column names

In [7]:
def clean_data(tmp_df):
    if 'Demised' in tmp_df.columns:
        tmp_df.rename(columns={'Demised':'Deaths'}, inplace=True)

    if 'Country/Region' in tmp_df.columns:
        tmp_df.rename(columns={'Country/Region':'country'}, inplace=True)
    
    if 'Province/State' in tmp_df.columns:
        tmp_df.rename(columns={'Province/State':'province'}, inplace=True)
      
    if 'Last Update' in tmp_df.columns:
        tmp_df.rename(columns={'Last Update':'date'}, inplace=True)
        
    if 'Suspected' in tmp_df.columns:
        tmp_df = tmp_df.drop(columns='Suspected')

    for col in tmp_df.columns:
        tmp_df[col] = tmp_df[col].fillna(0)
    
    #Lower case all col names
    tmp_df.columns = map(str.lower, tmp_df.columns)    
    
    return tmp_df

cleaned_dataframes = []
for frame in df_list:
    cleaned_dataframes.append(clean_data(frame))
    

In [9]:
#Impute the missing columns in the early stages with 0 values (recovered and deaths)
cleaned_dataframes[-1]['recovered'] = [0] * (cleaned_dataframes[-1]).shape[0]
cleaned_dataframes[-1]['deaths'] = [0] * (cleaned_dataframes[-1]).shape[0]

cleaned_dataframes[-2]['recovered'] = [0] * (cleaned_dataframes[-2]).shape[0]
cleaned_dataframes[-2]['deaths'] = [0] * (cleaned_dataframes[-2]).shape[0]

final_df = pd.concat(cleaned_dataframes, sort=True)

In [13]:
final_df['date'] = final_df['date'].astype(str)
final_df.head()

,confirmed,country,date,deaths,province,recovered
37,1,South Korea,1/22/2020 12:00,0,,0
15,0,China,1/22/2020 12:00,0,Inner Mongolia,0
14,4,China,1/22/2020 12:00,0,Hunan,0
13,444,China,1/22/2020 12:00,0,Hubei,0
12,,China,1/22/2020 12:00,0,Hong Kong,0


In [12]:
final_df = final_df.sort_values('date')

In [9]:
# ! pip install fastparquet

In [14]:
from datetime import datetime
import pyarrow

In [ ]:
file_name = './data/updated_{}.parquet.gzip'.format(datetime.date(datetime.now()))
final_df.to_parquet(file_name, engine='fastparquet', compression='gzip')

In [21]:
# test_csv = pd.read_csv(file_name)



In [22]:
test_csv

,confirmed,country,date,deaths,province,recovered
0,1.0,South Korea,1/22/2020 12:00,0.0,NaN,0.0
1,0.0,China,1/22/2020 12:00,0.0,Inner Mongolia,0.0
2,4.0,China,1/22/2020 12:00,0.0,Hunan,0.0
3,444.0,China,1/22/2020 12:00,0.0,Hubei,0.0
4,NaN,China,1/22/2020 12:00,0.0,Hong Kong,0.0
5,5.0,China,1/22/2020 12:00,0.0,Henan,0.0
6,NaN,China,1/22/2020 12:00,0.0,Heilongjiang,0.0
7,1.0,China,1/22/2020 12:00,0.0,Hebei,0.0
8,1.0,China,1/22/2020 12:00,0.0,Jiangsu,0.0
9,4.0,China,1/22/2020 12:00,0.0,Hainan,0.0
